In [63]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

df=pd.read_csv('./nasa.csv')

In [64]:
df['Est Dia']=(df['Est Dia in KM(max)']+df['Est Dia in KM(min)'])/2
df['Hazardous'].replace(True, 1, inplace=True)
df['Hazardous'].replace(False, 0, inplace=True)

df.drop(df.iloc[:, [0,1,5,6,7,8,9,10,13,16,17,19]], axis=1, inplace=True)
df.drop(df.iloc[:, [1,2]], axis=1, inplace=True)
df.drop(['Orbiting Body'], axis=1, inplace=True)
df.drop(['Equinox'], axis=1, inplace=True)

In [65]:
years=[]
for i in df['Close Approach Date'].values:
    years.extend(re.findall('\d+', i))
data=[]
for i in range(14061):
    if len(years[i])>3:
        data.append(years[i]) 
df['Close Approach Date']=data
df['Close Approach Date']=df['Close Approach Date'].astype(int)

years2=[]
for i in df['Orbit Determination Date'].values:
    years2.extend(re.findall('\d+', i))
data2=[]
for i in range(28121):
    if len(years2[i])==4:
        data2.append(years2[i])
df['Orbit Determination Date']=data2
df['Orbit Determination Date']=df['Orbit Determination Date'].astype(int)

In [66]:
scale1=df['Close Approach Date'].values.reshape(-1, 1)
scaler=LabelEncoder()
scaler.fit(scale1)
scaled1=scaler.transform(scale1)

scale2=df['Orbit ID'].values.reshape(-1, 1)
scaler=LabelEncoder()
scaler.fit(scale2)
scaled2=scaler.transform(scale2)

scale3=df['Orbit Determination Date'].values.reshape(-1, 1)
scaler=LabelEncoder()
scaler.fit(scale3)
scaled3=scaler.transform(scale3)

scale4=df['Orbit Uncertainity'].values.reshape(-1, 1)
scaler=LabelEncoder()
scaler.fit(scale4)
scaled4=scaler.transform(scale4)

scaled1=pd.DataFrame(scaled1.reshape(-1, 1), columns=['scaled_Cloase Approach Date'])
scaled2=pd.DataFrame(scaled2.reshape(-1, 1), columns=['scaled_Orbit ID'])
scaled3=pd.DataFrame(scaled3.reshape(-1, 1), columns=['scaled_Orbit Determination Date'])
scaled4=pd.DataFrame(scaled4.reshape(-1, 1), columns=['scaled_Orbit Uncertainity'])

ndf=pd.concat([df, scaled1, scaled2, scaled3, scaled4], axis=1)
ndf.drop(['Close Approach Date'], axis=1, inplace=True)
ndf.drop(['Orbit ID'], axis=1, inplace=True)
ndf.drop(['Orbit Determination Date'], axis=1, inplace=True)
ndf.drop(['Orbit Uncertainity'], axis=1, inplace=True)

In [67]:
y=ndf['Hazardous']
X=ndf.drop(['Hazardous'], axis=1, inplace=False)

scaler=StandardScaler()
scaler.fit(X)
X_scaled=scaler.transform(X)

X_train, X_test, y_train, y_test=train_test_split(X_scaled, y, test_size=0.2, random_state=156)

In [68]:
def get_clf_eval(y_test,pred=None, pred_proba=None):
    accuracy = accuracy_score(y_test,pred)
    confusion = confusion_matrix(y_test,pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test,pred)
    f1=f1_score(y_test, pred)
    roc_auc=roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    print("정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, f1:{3:.4f}, auc:{4:.4f}".format(accuracy,precision,recall,f1, roc_auc))

In [69]:
rf_clf=RandomForestClassifier()

rf_clf.fit(X_train, y_train)
preds=rf_clf.predict(X_test)
pred_proba=rf_clf.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, preds, pred_proba)

오차 행렬
[[787   0]
 [  5 146]]
정확도 : 0.9947, 정밀도 : 1.0000, 재현율 : 0.9669, f1:0.9832, auc:0.9982


In [70]:
print('hi')

hi
